In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import requests
import os


os.environ["SERP_API_KEY"] = "Your_serp_API_key"
SERP_API_KEY = os.getenv("SERP_API_KEY")


def serpapi_search(query):
    url = "https://serpapi.com/search"
    params = {
        "q": query,
        "api_key": SERP_API_KEY,
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def get_search_results(query):
    results = serpapi_search(query)
    search_results = results.get("organic_results", [])

    if not search_results:
        return "No results found."


    search_info = []
    for result in search_results:
        title = result.get('title', 'No title')
        link = result.get('link', '#')
        snippet = result.get('snippet', 'No description available')
        search_info.append(f"<b>{title}</b>: {snippet} <br><a href='{link}' target='_blank'>{link}</a>")

    return "<br><br>".join(search_info)


def search_company_industry(company_name):
    query = f"{company_name} industry or business sector"
    results = get_search_results(query)

    if not results:
        return "Industry not found"

    return results.split("<br>")[0]


def search_for_datasets(query):
    kaggle_query = f"{query} site:kaggle.com"
    huggingface_query = f"{query} site:huggingface.co"
    github_query = f"{query} site:github.com"

    kaggle_results = get_search_results(kaggle_query)
    huggingface_results = get_search_results(huggingface_query)
    github_results = get_search_results(github_query)

    kaggle_heading = "<span style='font-size:30px;'>Kaggle Datasets:</span><br>"
    huggingface_heading = "<span style='font-size:30px;'>HuggingFace Datasets:</span><br>"
    github_heading = "<span style='font-size:30px;'>GitHub Datasets:</span><br>"

    dataset_results = f"<b>Relevant Datasets</b><br>{kaggle_heading}{kaggle_results}<br><br>{huggingface_heading}{huggingface_results}<br><br>{github_heading}{github_results}"

    return dataset_results

def increase_font_size(text, font_size="24px"):
    return f"<span style='font-size:{font_size};'>{text}</span>"


class MarketResearchCrew:
    def __init__(self, company_name=None):
        self.company_name = company_name
        self.industry = self.auto_determine_industry()

    def auto_determine_industry(self):
        return search_company_industry(self.company_name)

    def research_agent(self):
        query = f"{self.company_name} company overview" if self.company_name else f"{self.industry} industry trends"
        results = get_search_results(query)
        return f"{increase_font_size('<b>Research Agent Results</b>', '30px')}<br>{results}"

    def market_analysis_agent(self):
        query = f"{self.industry} industry AI ML automation trends"
        results = get_search_results(query)
        return f"{increase_font_size('<b>Market Analysis Agent Results</b>', '30px')}<br>{results}"

    def use_case_generation_agent(self):
        query1 = f"AI ML automation use cases in {self.industry}, case studies, and industry applications"
        results1 = get_search_results(query1)

        query2 = f"Top AI ML applications in {self.company_name}"
        results2 = get_search_results(query2)

        full_results = f"{increase_font_size('<b>Industry Use Cases</b>', '30px')}<br>{results1}<br><br>{increase_font_size('<b>Company Use Cases</b>', '30px')}<br>{results2}"
         return f"{increase_font_size('<b>Use Case Generation Agent Results</b>', '30px')}<br>{full_results}"

    def dataset_agent(self):
        dataset_results = search_for_datasets(self.industry)
        return f"{increase_font_size('<b>Relevant Datasets</b>', '30px')}<br>{dataset_results}"

    def run_crew(self):
        research_report = self.research_agent()
        market_analysis_report = self.market_analysis_agent()
        use_case_report = self.use_case_generation_agent()
        dataset_report = self.dataset_agent()

        full_report = (
            f"<center><h1>Market Research Report</h1></center>"
            f"<h2>Company: {self.company_name}</h2>"
            f"<h2>Industry: {self.industry}</h2><br>"
            f"{research_report}<br><br>"
            f"{market_analysis_report}<br><br>"
            f"{use_case_report}<br><br>"
            f"{dataset_report}"
        )

        return full_report

def generate_report(company_name):
    crew = MarketResearchCrew(company_name=company_name)
    report = crew.run_crew()
    return report

interface = gr.Interface(
    fn=generate_report,
    inputs=gr.Textbox(label="Enter Company Name"),
    outputs=gr.HTML(label="Market Research Report"),
    title="Market Research Generator",
    description="Enter the company name to generate a detailed market research report. Clickable links included in the output.",
)

if __name__ == "__main__":
    interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://be9c6a1465208f1218.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
